In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate

Ejercicios: Álgebra lineal

Problemas: 3,4,5,6,7,9,10,12,13,14,15

## Punto 3. Algoritmo para mutiplicar dos matrices

In [22]:
def MultiplyMatrix(matrix1, matrix2):
    if len(matrix1[0])!=len(matrix2):
        return 'No es posible realizar el producto.'
    else:
        result_matrix = np.zeros([len(matrix1),len(matrix2[0])])
        for i in range(len(matrix1)):
            for j in range(len(matrix2[0])):
                for k in range(len(matrix2)):
                    result_matrix[i][j] += matrix1[i][k]*matrix2[k][j]
        return result_matrix

In [24]:
M1 = np.array([[1, 0, 0],[5, 1, 0],[-2, 3, 1]])
M2 = np.array([[4, -2, 1],[0, 3, 7],[0, 0, 2]])

MultiplyMatrix(M1,M2)

array([[ 4., -2.,  1.],
       [20., -7., 12.],
       [-8., 13., 21.]])